# Boston Housing Dataset: Data Preprocessing and Exploratory Data Analysis

This notebook focuses on the initial data exploration, preprocessing, and exploratory data analysis of the Boston Housing dataset.

## 1. Import Libraries

In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization styles
plt.style.use('seaborn-whitegrid')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Display all columns
pd.set_option('display.max_columns', None)

## 2. Load and Inspect the Dataset

In [ ]:
# Load the dataset
df = pd.read_csv('bostonHousing.csv')

# Display the first few rows
print("First 5 rows of the dataset:")
df.head()

In [ ]:
# Check the shape of the dataset
print(f"Dataset shape: {df.shape}")

# Get column information
print("\nColumn information:")
df.info()

In [ ]:
# Statistical summary
print("Statistical summary of the dataset:")
df.describe().T

## 3. Data Preprocessing

### 3.1 Handling Missing Values

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

# Visualize missing values if any
if missing_values.sum() > 0:
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
    plt.title('Missing Values Heatmap')
    plt.show()
else:
    print("\nNo missing values found in the dataset!")

### 3.2 Handling Outliers

Let's identify outliers using box plots and the Interquartile Range (IQR) method.

In [ ]:
# Create box plots for all numeric features
plt.figure(figsize=(15, 10))
df.boxplot()
plt.title('Box Plots for All Features')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Function to detect outliers using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Detect outliers for each numeric column
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

for column in numeric_columns:
    outliers, lower, upper = detect_outliers_iqr(df, column)
    if len(outliers) > 0:
        print(f"\nOutliers in {column}:")
        print(f"Number of outliers: {len(outliers)}")
        print(f"Lower bound: {lower:.2f}, Upper bound: {upper:.2f}")
        print(f"Min value: {df[column].min():.2f}, Max value: {df[column].max():.2f}")

### 3.3 Feature Scaling and Normalization

In [ ]:
# Import scaling libraries
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# Create a copy of the dataframe for scaling
df_scaled = df.copy()

# Separate target variable
X = df_scaled.drop(['MEDV', 'CAT. MEDV'], axis=1)
y = df_scaled['MEDV']

# Apply different scaling techniques
# 1. Standard Scaling (Z-score normalization)
scaler_standard = StandardScaler()
X_standard = scaler_standard.fit_transform(X)
X_standard_df = pd.DataFrame(X_standard, columns=X.columns)

# 2. Min-Max Scaling
scaler_minmax = MinMaxScaler()
X_minmax = scaler_minmax.fit_transform(X)
X_minmax_df = pd.DataFrame(X_minmax, columns=X.columns)

# 3. Robust Scaling (less sensitive to outliers)
scaler_robust = RobustScaler()
X_robust = scaler_robust.fit_transform(X)
X_robust_df = pd.DataFrame(X_robust, columns=X.columns)

# Compare the distributions after scaling
fig, axes = plt.subplots(4, 1, figsize=(15, 20))

# Original data
sns.boxplot(data=X, ax=axes[0])
axes[0].set_title('Original Data')
axes[0].set_xticklabels(X.columns, rotation=90)

# Standard scaled data
sns.boxplot(data=X_standard_df, ax=axes[1])
axes[1].set_title('Standard Scaled Data')
axes[1].set_xticklabels(X.columns, rotation=90)

# Min-Max scaled data
sns.boxplot(data=X_minmax_df, ax=axes[2])
axes[2].set_title('Min-Max Scaled Data')
axes[2].set_xticklabels(X.columns, rotation=90)

# Robust scaled data
sns.boxplot(data=X_robust_df, ax=axes[3])
axes[3].set_title('Robust Scaled Data')
axes[3].set_xticklabels(X.columns, rotation=90)

plt.tight_layout()
plt.show()

### 3.4 Feature Engineering

Let's create some new features that might be useful for our regression task.

In [ ]:
# Create a copy for feature engineering
df_fe = df.copy()

# 1. Log transformation for skewed features
skewed_features = ['CRIM', 'ZN', 'DIS', 'LSTAT']
for feature in skewed_features:
    df_fe[f'{feature}_log'] = np.log1p(df_fe[feature])

# 2. Polynomial features for important predictors
poly_features = ['RM', 'LSTAT', 'DIS']
for feature in poly_features:
    df_fe[f'{feature}_squared'] = df_fe[feature] ** 2

# 3. Interaction terms
df_fe['RM_LSTAT'] = df_fe['RM'] * df_fe['LSTAT']
df_fe['NOX_INDUS'] = df_fe['NOX'] * df_fe['INDUS']
df_fe['RM_AGE'] = df_fe['RM'] / (df_fe['AGE'] + 1)  # Adding 1 to avoid division by zero

# Display the new features
print("New features created:")
new_features = [col for col in df_fe.columns if col not in df.columns]
df_fe[new_features].head()

## 4. Exploratory Data Analysis (EDA)

### 4.1 Distribution of Target Variable

In [ ]:
# Visualize the distribution of the target variable (MEDV)
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.histplot(df['MEDV'], kde=True)
plt.title('Distribution of MEDV (Median Home Value)')
plt.xlabel('MEDV ($1000s)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
sns.boxplot(y=df['MEDV'])
plt.title('Box Plot of MEDV')
plt.ylabel('MEDV ($1000s)')

plt.tight_layout()
plt.show()

In [ ]:
# Check for skewness in the target variable
skewness = df['MEDV'].skew()
kurtosis = df['MEDV'].kurt()

print(f"Skewness of MEDV: {skewness:.4f}")
print(f"Kurtosis of MEDV: {kurtosis:.4f}")

# If skewness is significant, apply log transformation
if abs(skewness) > 0.5:
    df['MEDV_log'] = np.log1p(df['MEDV'])
    
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    sns.histplot(df['MEDV'], kde=True)
    plt.title('Original MEDV Distribution')
    
    plt.subplot(1, 2, 2)
    sns.histplot(df['MEDV_log'], kde=True)
    plt.title('Log-Transformed MEDV Distribution')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Skewness after log transformation: {df['MEDV_log'].skew():.4f}")

### 4.2 Correlation Analysis

In [ ]:
# Calculate correlation matrix
correlation_matrix = df.corr()

# Plot heatmap of correlations
plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix of Boston Housing Features')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation with target variable
target_corr = correlation_matrix['MEDV'].sort_values(ascending=False)
print("Correlation with MEDV (target variable):")
print(target_corr)

# Visualize top correlations with target
plt.figure(figsize=(12, 8))
target_corr.drop('MEDV').plot(kind='bar')
plt.title('Feature Correlation with MEDV')
plt.xlabel('Features')
plt.ylabel('Correlation Coefficient')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 4.3 Multicollinearity Check

In [ ]:
# Check for multicollinearity using Variance Inflation Factor (VIF)
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Create a dataframe for VIF calculation
X_vif = df.drop(['MEDV', 'CAT. MEDV'], axis=1)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

# Sort by VIF value
vif_data = vif_data.sort_values(by="VIF", ascending=False)
print("Variance Inflation Factor (VIF) for features:")
print(vif_data)

# Visualize VIF values
plt.figure(figsize=(12, 8))
sns.barplot(x="VIF", y="Feature", data=vif_data)
plt.title('Variance Inflation Factor (VIF) for Features')
plt.axvline(x=5, color='red', linestyle='--', label='VIF=5 (Threshold for high multicollinearity)')
plt.axvline(x=10, color='darkred', linestyle='--', label='VIF=10 (Threshold for severe multicollinearity)')
plt.legend()
plt.tight_layout()
plt.show()

### 4.4 Feature Visualization

In [ ]:
# Visualize the distribution of each feature
plt.figure(figsize=(20, 15))
for i, column in enumerate(df.drop(['MEDV', 'CAT. MEDV'], axis=1).columns):
    plt.subplot(4, 4, i+1)
    sns.histplot(df[column], kde=True)
    plt.title(f'Distribution of {column}')
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots of top correlated features with MEDV
top_corr_features = target_corr.drop('MEDV').abs().sort_values(ascending=False).head(4).index

plt.figure(figsize=(15, 10))
for i, feature in enumerate(top_corr_features):
    plt.subplot(2, 2, i+1)
    sns.scatterplot(x=df[feature], y=df['MEDV'])
    plt.title(f'MEDV vs {feature}')
    plt.xlabel(feature)
    plt.ylabel('MEDV')
    
    # Add regression line
    sns.regplot(x=df[feature], y=df['MEDV'], scatter=False, color='red')
    
plt.tight_layout()
plt.show()

### 4.5 Pair Plots for Key Features

In [ ]:
# Select important features based on correlation
important_features = list(top_corr_features) + ['MEDV']
subset_df = df[important_features]

# Create pair plot
sns.pairplot(subset_df, diag_kind='kde')
plt.suptitle('Pair Plot of Key Features', y=1.02)
plt.tight_layout()
plt.show()

## 5. Summary of Findings

### Data Preprocessing:
1. **Missing Values**: We checked for missing values in the dataset.
2. **Outliers**: We identified outliers using box plots and the IQR method.
3. **Feature Scaling**: We applied different scaling techniques (Standard, MinMax, Robust) to normalize the features.
4. **Feature Engineering**: We created new features through log transformations, polynomial features, and interaction terms.

### Exploratory Data Analysis:
1. **Target Variable Distribution**: We analyzed the distribution of the target variable (MEDV) and applied transformations if needed.
2. **Correlation Analysis**: We identified the features most correlated with the target variable.
3. **Multicollinearity**: We checked for multicollinearity among features using VIF.
4. **Feature Visualization**: We visualized the distribution of features and their relationships with the target variable.

### Key Insights:
- The most important features for predicting median home values appear to be [will be filled based on actual analysis].
- Some features show high multicollinearity, which might affect model performance.
- The target variable distribution shows [will be filled based on actual analysis].
- Feature engineering, especially [will be filled based on actual analysis], might improve model performance.